# Set Covering - 2023-10-10
Copyright(c) 2023 Alex Buffa


In [103]:
import numpy as np
from random import random
from typing import Tuple, Set
from functools import reduce
from operator import or_
from queue import PriorityQueue, LifoQueue, SimpleQueue, Queue
from collections import namedtuple
from typing import Callable
Result = namedtuple("Result", ["name", "iters", "state", "coverage", "prio"])
State = Tuple[Set[int], Set[int]]

Define our problem data

In [104]:
PROBLEM_SIZE = 8
NUM_SETS = 10
THRESHOLD = 0.3
SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
# Redefine SETS until the problem is solvable
while not all(reduce(or_, [SETS[i] for i in range(NUM_SETS)])):
    SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
results: dict[str, Result] = dict()


In [105]:
# Utility function just to see our current taken array
def visualize_state(state: State) -> list[int]:
    return sum([SETS[i] for i in state[0]])

In [106]:
def goal_check(state: State):
    return all(reduce(or_, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)])))

In [107]:
def search(name: str, initial_state: State = None,*, frontier: "Queue" = None, priority: Callable[[State],int] = None) -> Result:
    """Generic Search Function.
    Through the parameters 
    """
    if initial_state is None:
        initial_state = (set(), set(range(NUM_SETS)))
    assert len(initial_state) == 2, "Invalid State"
    if frontier is None:
        frontier = PriorityQueue()
    if priority is None:
        priority = lambda _: None
    WrappedState = namedtuple("WrappedState", ["priority", "state"])
    frontier.put(WrappedState(priority(initial_state), initial_state))
    _, state = frontier.get()
    counter = 0
    while not goal_check(state):
        counter += 1
        for a in state[1]:
            new_state = (state[0] ^ {a}, state[1] ^ {a})
            frontier.put(WrappedState(priority(new_state), new_state))
        _, state = frontier.get()
    res = Result(name, counter, state, visualize_state(state), priority(state))
    results[name] = res
    return res


Depth First Search

In [108]:
search(name="Depth-First", frontier=LifoQueue()).state

({1, 2, 3, 4, 5, 6, 7, 8, 9}, {0})

Breadth First Search

In [109]:
# Using SimpleQueue, which does it internally
search(name="Breadth-First", frontier=SimpleQueue()).state

({0, 2, 4}, {1, 3, 5, 6, 7, 8, 9})

I now define a function to measure a cost of a given state, based only on the actions done in the past.  
Doing so we are approaching the problem with an *uninformed* approach.

In [110]:
def uninformed_cost(state: State) -> int:
    """Number of sets"""
    return len(state[0])


In [111]:
search(name="Djikstra", priority=uninformed_cost).state

({0, 4, 7}, {1, 2, 3, 5, 6, 8, 9})

We try now the *informed* approach by defining a cost function that takes into account the distance from the goal.  
For example, we define the distance function as the number of nodes that are not yet covered.

In [112]:
def distance(state: State) -> int:
    if(len(state[0]) == 0 ):
        return PROBLEM_SIZE
    return (sum([SETS[i] for i in state[0]]) == 0).sum()

A* requires a heuristic function that is admissible, i.e. it never overestimates the cost to reach the goal.  
With the above distance function we have an admissible heuristic function.  
The priority for A* is given by the sum of the uninformed cost function and the heuristic function.

In [113]:
search(name="A*", priority=lambda x: uninformed_cost(x) + distance(x)).state

({0, 2, 4}, {1, 3, 5, 6, 7, 8, 9})

In [114]:
print("All the results obtained above, sorted by number of iterations")
for result in sorted(results.values(), key=lambda x: x.iters):
    print(result)

All the results obtained above, sorted by number of iterations
Result(name='A*', iters=3, state=({0, 2, 4}, {1, 3, 5, 6, 7, 8, 9}), coverage=array([1, 1, 1, 1, 1, 1, 1, 1]), prio=3)
Result(name='Depth-First', iters=9, state=({1, 2, 3, 4, 5, 6, 7, 8, 9}, {0}), coverage=array([1, 3, 4, 2, 4, 3, 1, 1]), prio=None)
Result(name='Breadth-First', iters=111, state=({0, 2, 4}, {1, 3, 5, 6, 7, 8, 9}), coverage=array([1, 1, 1, 1, 1, 1, 1, 1]), prio=None)
Result(name='Djikstra', iters=174, state=({0, 4, 7}, {1, 2, 3, 5, 6, 8, 9}), coverage=array([1, 2, 1, 1, 1, 1, 1, 1]), prio=3)
